In [16]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# 중력 가속도 (m/s²)
GRAVITY = 9.81  

# Inertial Signals 데이터 로드 함수
def load_inertial_data():
    """
    UCI HAR Inertial Signals 데이터를 로드하는 함수.
    
    출력:
        X_df (DataFrame): 가속도 데이터 (X, Y, Z)
        y_df (DataFrame): 활동 라벨
    """
    data_dir = "E:/dataset/HAR/UCI-HAR/train/Inertial Signals/"

    # Inertial Signals 가속도 데이터 로드
    acc_x = np.loadtxt(os.path.join(data_dir, "body_acc_x_train.txt"))
    acc_y = np.loadtxt(os.path.join(data_dir, "body_acc_y_train.txt"))
    acc_z = np.loadtxt(os.path.join(data_dir, "body_acc_z_train.txt"))

    # 활동 라벨 로드
    y_path = "E:/dataset/HAR/UCI-HAR/train/y_train.txt"
    y_df = pd.read_csv(y_path, delim_whitespace=True, header=None, names=["Activity"])

    # 각 샘플의 평균 가속도 값 사용 (샘플당 128개의 시퀀스 값 존재)
    X_df = pd.DataFrame({
        "ax": np.mean(acc_x, axis=1),
        "ay": np.mean(acc_y, axis=1),
        "az": np.mean(acc_z, axis=1),
    })

    return X_df, y_df

# 데이터 로드
X_df, y_df = load_inertial_data()

# 활동 라벨 매핑
activity_labels = {
    1: "걷기",
    2: "계단 오르기",
    3: "계단 내리기",
    4: "앉음",
    5: "서있음",
    6: "누움"
}
y_df["Activity"] = y_df["Activity"].map(activity_labels)

# 정적 행위만 필터링
static_activities = ["앉음", "서있음", "누움"]
filtered_indices = y_df["Activity"].isin(static_activities)
X_df = X_df[filtered_indices].reset_index(drop=True)
y_df = y_df[filtered_indices].reset_index(drop=True)

# 특징(feature) 계산 함수
def calculate_static_features(ax, ay, az):
    """
    정적 행위에서 다양한 특징을 계산.
    """
    features = {}

    # Theta (기울기 각도)
    az_norm = az / GRAVITY  # 중력 가속도로 정규화
    if -1 <= az_norm <= 1:
        features["Theta"] = np.degrees(np.arccos(az_norm))
    else:
        features["Theta"] = np.nan

    # XY 평면 기울기 (Tilt Angle in XY Plane)
    features["Theta_XY"] = np.degrees(np.arctan2(ay, ax))

    # 가속도 벡터 크기 (Magnitude)
    features["Magnitude"] = np.sqrt(ax**2 + ay**2 + az**2)

    # Azimuth 및 Elevation 계산
    features["Azimuth"] = np.degrees(np.arctan2(ay, ax))
    features["Elevation"] = np.degrees(np.arctan2(np.sqrt(ax**2 + ay**2), az))

    return features

# 모든 샘플에 대해 특징 계산
features_data = X_df.apply(lambda row: calculate_static_features(row["ax"], row["ay"], row["az"]), axis=1)
features_df = pd.DataFrame(features_data.tolist())

# 기존 데이터와 결합
X_df = pd.concat([X_df.reset_index(drop=True), features_df], axis=1)

# 정적 행위 임계값 기반 분류 함수
def classify_static_activity(row):
    """ Theta, Theta_XY, Magnitude, Azimuth, Elevation을 기반으로 정적 행위 분류 """
    theta = row["Theta"]
    elevation = row["Elevation"]
    magnitude = row["Magnitude"]

    # NaN 값 체크
    if np.isnan(theta) or np.isnan(elevation) or np.isnan(magnitude):
        return "미확인"

    # 조건 기반 분류
    if theta < 30 and magnitude > 9.5:
        return "서있음"
    elif 30 <= theta < 60 and elevation < 45:
        return "앉음"
    else:
        return "누움"

# 정적 행위 판정 적용
X_df["Predicted_Activity"] = X_df.apply(classify_static_activity, axis=1)

# 결과 비교
comparison = pd.DataFrame({
    "Actual": y_df["Activity"].values,
    "Predicted": X_df["Predicted_Activity"].values,
    "Theta": X_df["Theta"].values,
    "Theta_XY": X_df["Theta_XY"].values,
    "Magnitude": X_df["Magnitude"].values,
    "Azimuth": X_df["Azimuth"].values,
    "Elevation": X_df["Elevation"].values
})

# 정확도 계산
accuracy = (comparison["Actual"] == comparison["Predicted"]).mean()
print(f"정적 행위 인식 정확도: {accuracy:.2%}")


정적 행위 인식 정확도: 34.60%


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17204\336291269.py:28: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y_df = pd.read_csv(y_path, delim_whitespace=True, header=None, names=["Activity"])


In [18]:
X_df

,ax,ay,az,Theta,Theta_XY,Magnitude,Azimuth,Elevation,Predicted_Activity
0,0.002269,-0.001655,-0.008126,90.047458,-36.116712,0.008597,-36.116712,160.933417,누움
1,0.000174,0.000382,-0.005025,90.029351,65.522195,0.005043,65.522195,175.230255,누움
2,0.000428,-0.001222,-0.001703,90.009944,-70.687635,0.002139,-70.687635,142.755380,누움
3,0.000329,-0.004753,-0.004947,90.028892,-86.036435,0.006868,-86.036435,136.073834,누움
4,-0.000195,0.000298,-0.002330,90.013610,123.204042,0.002357,123.204042,171.305119,누움
...,...,...,...,...,...,...,...,...,...
4062,0.002394,-0.002355,0.003110,89.981839,-44.538429,0.004577,-44.538429,47.200004,누움
4063,0.002326,-0.002249,0.002272,89.986732,-44.038088,0.003954,-44.038088,54.929468,누움
4064,0.000996,-0.000479,-0.000217,90.001269,-25.676080,0.001126,-25.676080,101.121056,누움
4065,0.001071,0.000576,-0.000986,90.005759,28.279051,0.001565,28.279051,129.044990,누움


In [22]:
y_df

,Activity
0,서있음
1,서있음
2,서있음
3,서있음
4,서있음
...,...
4062,누움
4063,누움
4064,누움
4065,누움


In [21]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 모든 샘플에 대해 특징 계산
features_data = X_df.apply(lambda row: calculate_static_features(row["ax"], row["ay"], row["az"]), axis=1)
features_df = pd.DataFrame(features_data.tolist())

# 기존 데이터와 결합
X_df = pd.concat([X_df.reset_index(drop=True), features_df], axis=1)

# 랜덤 포레스트를 사용한 행위 분류
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df["Activity"], test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"랜덤 포레스트 기반 정적 행위 인식 정확도: {accuracy:.2%}")



ValueError: could not convert string to float: '누움'